In [5]:
lang = 'als'
model_idx = 'gbert'

path = 'bitext/awesome_align/'
output_path = 'bitext/bli/'


sep = '<sep>'

In [6]:
import pandas as pd
import os 
from collections import Counter 

import nltk
german_stop_words = nltk.corpus.stopwords.words('german')

import string
punct = list(string.punctuation)

if not os.path.exists(output_path):
    os.mkdir(output_path)
    
import editdistance

from tqdm import tqdm
tqdm.pandas()

import numpy as np

In [7]:
word_fname = f'{path}/{model_idx}_{lang}de.word'
prob_gname = f'{path}/{model_idx}_{lang}de.prob'
word_lines = open(word_fname).readlines()
prob_lines = open(prob_gname).readlines()
word_lines[0],prob_lines[0]

('Kult<sep>Kult .<sep>.\n', '1.0 1.0\n')

In [8]:
len(word_lines)

160611

In [185]:
records = []

for words, probs in zip(word_lines, prob_lines):
    tokens = [token.split(sep) for token in words.lower().split()]
    vals = [float(v) for v in probs.split()]
    records += [[i[0][0], i[0][1], i[1]] for i in zip(tokens, vals)]
     
df = pd.DataFrame.from_records(records, columns = [lang, 'de', 'prob'])
df.sort_values(by="prob", inplace=True, ascending = False)
df = df.groupby([lang, 'de'])['prob'].apply(list).reset_index(name='prob')

,als,de,prob
0,!,!,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,!,",","[0.9656196236610413, 0.9656196236610413]"
2,!,.,[0.9986563920974731]
3,!,und,[0.44471311569213867]
4,"""",!,[0.980839729309082]


In [186]:
print(len(df))
same_words_idx = df[df[lang] == df['de']].index
df.drop(same_words_idx, axis=0, inplace=True)

stop_words_idx = df[df['de'].isin(german_stop_words)].index
df.drop(stop_words_idx, axis=0, inplace=True)

short_words_idx = df[df[lang].str.len() == 1].index
df.drop(short_words_idx, axis=0, inplace=True)

short_words_idx = df[df['de'].str.len() == 1].index
df.drop(short_words_idx, axis=0, inplace=True)

punct_idx = df[df['de'].isin(punct)].index
df.drop(punct_idx, axis=0, inplace=True)

number_idx = df[df['de'].str.contains('[0-9]+')].index
df.drop(number_idx, axis=0, inplace=True)

punct_idx = df[df[lang].isin(punct)].index
df.drop(punct_idx, axis=0, inplace=True)

number_idx = df[df[lang].str.contains('[0-9]+')].index
df.drop(number_idx, axis=0, inplace=True)

df['nchar'] = df['de'].apply(len)
uni_idx = df[df['nchar'] <= 2].index
df.drop(uni_idx, axis=0, inplace=True)

df['edit_distance'] = df.progress_apply(lambda x: editdistance.eval(x[lang], x['de']), axis=1)

df['freq'] = df['prob'].apply(len)
df['avg_prob'] = df['prob'].apply(np.mean)
df['std_prob'] = df['prob'].apply(np.std)

df.reset_index(inplace=True, drop=True)

df.to_csv(f'{output_path}/{model_idx}_{lang}.csv', index=None)
print(len(df))

534421


100%|███████████████████████████████| 379060/379060 [00:03<00:00, 112678.85it/s]


379060
